In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.3.1'

In [3]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [4]:
mnist = tf.keras.datasets.mnist

In [35]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [36]:
x_train, x_test = x_train/255.0, x_test/255.0

In [37]:
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [38]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)

In [39]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [40]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [41]:
model = MyModel()

In [42]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [43]:
optimizer = tf.keras.optimizers.Adam()

In [44]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [45]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [46]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)    

In [49]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training = False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [50]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.04488351196050644, Accuracy: 98.61166381835938, Test Loss: 0.0558326281607151, Test Accuracy: 98.02999877929688
Epoch 2, Loss: 0.0222530048340559, Accuracy: 99.3133316040039, Test Loss: 0.053055163472890854, Test Accuracy: 98.36000061035156
Epoch 3, Loss: 0.01481110230088234, Accuracy: 99.51666259765625, Test Loss: 0.061899710446596146, Test Accuracy: 98.4000015258789
Epoch 4, Loss: 0.008768548257648945, Accuracy: 99.70333862304688, Test Loss: 0.062264494597911835, Test Accuracy: 98.40999603271484
Epoch 5, Loss: 0.00795793067663908, Accuracy: 99.73500061035156, Test Loss: 0.06602773815393448, Test Accuracy: 98.25
